# ╬ DreamStreets: AI-Powered Query and Visualization of Street Networks using GPT-OSS-120b, NetworkX, and LangGraph
Author: Adam Munawar Rahman, September 2025

**OpenAI Open Model Hackathon Submission** - Extending AskStreets with GPT-OSS-120b's Advanced Reasoning

Using powerful open-source libraries like OSMnx and NetworkX, we can retrieve geographic features and street network datasets from OpenStreetMap and persist them as graphs in DuckDB. Then, via a LangGraph ReAct agent powered by GPT-OSS-120b, we feed natural language queries to AI-based tools to execute complex lookups, run advanced graph algorithms, and analyze geospatial data. This agentic app enables meaningful insights into the network properties of the desired geographic location, and empowers us to address real-world infrastructure challenges.

## What's New with GPT-OSS-120b
This implementation showcases OpenAI's most powerful open-source model (120 billion parameters) running completely offline. GPT-OSS-120b brings superior chain-of-thought reasoning capabilities, enabling more complex multi-step analysis and better code generation for graph algorithms. The model runs locally via Ollama, demonstrating true offline capability - crucial for humanitarian field work where internet connectivity is unreliable.

## Requirements
- Python 3.10+
- GPT-OSS-120b model loaded in Ollama
- DuckDB for spatial data operations
- NetworkX for graph algorithms
- No internet connection required after initial setup


# I. System Initialization and GPT-OSS Model Setup

We begin by initializing the GPT-OSS-120b model and setting up our environment. The model initialization is done once and reused throughout the session for optimal performance.

In [ ]:
#!/usr/bin/env python
"""
DreamStreets: AI-Powered Street Network Analysis using GPT-OSS-120b
Author: Adam Munawar Rahman, September 2025

Using GPT-OSS-120b's advanced reasoning capabilities, we can analyze street networks
and provide insights for urban planning and humanitarian response scenarios.
"""

# ============================================================================
# Import statements and global configuration
# ============================================================================

# Standard library imports
import json
import math
import os
import time
from pathlib import Path
from typing import Dict, Any, List

# Data manipulation and analysis libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# Database operations
import duckdb

# Network analysis libraries
import networkx as nx
import osmnx as ox

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# LangChain and LLM integration
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

# Rich console output for showcasing model reasoning
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich import box

# Jupyter display utilities
from IPython.display import display, Markdown as IPMarkdown, HTML

# ============================================================================
# Configure OSMnx settings
# ============================================================================

# Enable caching to avoid redundant API calls
ox.settings.use_cache = True
# Disable console logging for cleaner output
ox.settings.log_console = False

# ============================================================================
# Initialize global components
# ============================================================================

# Console for formatted output
console = Console()

# Global state management dictionary
# This holds our graph, database connection, and schema information
state = {
    'graph': None,        # NetworkX graph object
    'db': None,          # DuckDB connection
    'schema': {},        # Database and graph schema information
}

# CRITICAL: Single global LLM instance for performance optimization
# Initializing GPT-OSS-120b once prevents repeated model loading
llm = None

print("="*80)
print("DreamStreets System Initialization")
print("Using GPT-OSS-120b for advanced street network analysis")
print("="*80)

In [ ]:
def initialize_gpt_oss():
    """
    Initialize GPT-OSS-120b model for all analysis components.
    
    This function loads the 120 billion parameter model once and keeps it in memory
    for the entire session. The model runs completely offline via Ollama, demonstrating
    the "Best Local Agent" capability for the hackathon.
    
    Returns:
        ChatOllama: Initialized LLM instance ready for analysis
    """
    global llm
    
    # Check if already initialized to avoid redundant loading
    if llm is not None:
        return llm
    
    console.print(Panel.fit(
        "[bold cyan]▶ Initializing GPT-OSS-120b[/bold cyan]\n"
        "[dim]OpenAI's most powerful open-source model with advanced chain-of-thought reasoning[/dim]\n"
        "[dim]This one-time setup takes ~30 seconds but ensures optimal performance[/dim]",
        border_style="cyan",
        box=box.ROUNDED
    ))
    
    print("\nLoading GPT-OSS-120b model weights...")
    print("Model size: 120 billion parameters")
    print("Running mode: Completely offline via Ollama")
    
    # Create single LLM instance with low temperature for consistent reasoning
    llm = ChatOllama(model="gpt-oss:120b", temperature=0.1)
    
    # Warm up the model with an initial query
    # This loads the model into memory and prepares it for analysis
    with console.status("[bold green]Loading model weights and initializing reasoning engine..."):
        warmup_response = llm.invoke("Initialize chain-of-thought reasoning for street network analysis.")
        print(f"Model initialization complete: {len(warmup_response.content)} tokens generated")
    
    console.print("[bold green]✓ GPT-OSS-120b ready for advanced analysis![/bold green]\n")
    print(f"Model successfully loaded and warmed up")
    print("-"*80)
    
    return llm

# Initialize the model once at notebook start
# This prevents repeated loading during tool execution
print("\nInitializing GPT-OSS-120b for the session...")
llm = initialize_gpt_oss()
print("\nModel initialization complete. Ready for analysis.")

# ============================================================================
# II. Loading Street Network Data and Database Setup
# ============================================================================

Next, we load the street network graph from GraphML format and set up our DuckDB spatial database. The graph represents intersections as nodes and street segments as edges. We also handle the critical data type conversions needed for NetworkX algorithms to function properly.

In [ ]:
def initialize_environment(graphml_path: str = 'chinatown.graphml', db_path: str = 'chinatown.duckdb'):
    """
    Initialize the street network graph and spatial database for analysis.
    
    This function performs several critical operations:
    1. Loads the street network graph from GraphML format
    2. Converts string attributes to proper numeric types (CRITICAL for NetworkX)
    3. Connects to DuckDB with spatial extensions
    4. Builds schema information for query generation
    
    Args:
        graphml_path: Path to the GraphML file containing street network
        db_path: Path to the DuckDB database file
    
    Returns:
        bool: True if initialization successful
    """
    
    print("\n" + "="*80)
    print(f"Loading Street Network Environment")
    print(f"Graph file: {graphml_path}")
    print(f"Database file: {db_path}")
    print("="*80)
    
    console.print(f"\n[bold]◉ Loading Street Network: {graphml_path}[/bold]")
    
    try:
        # ============================================================================
        # Load the street network graph from GraphML
        # ============================================================================
        print("\nStep 1: Loading graph from GraphML file...")
        state['graph'] = nx.read_graphml(graphml_path)
        print(f"Graph loaded successfully")
        
        # ============================================================================
        # CRITICAL FIX: Convert string attributes to numeric types
        # GraphML loads numeric values as strings, which breaks NetworkX algorithms
        # This is essential for distance calculations and centrality metrics
        # ============================================================================
        console.print("[dim]Converting string attributes to numeric types...[/dim]")
        print("\nStep 2: Converting attribute data types...")
        
        # Convert edge attributes
        edge_conversions = 0
        for u, v, data in state['graph'].edges(data=True):
            for key, value in data.items():
                if isinstance(value, str) and key in ['length', 'travel_time']:
                    try:
                        data[key] = float(value)
                        edge_conversions += 1
                    except (ValueError, TypeError):
                        # Some attributes may not be convertible, skip them
                        pass
        
        # Convert node attributes
        node_conversions = 0
        for node, data in state['graph'].nodes(data=True):
            for key, value in data.items():
                if isinstance(value, str) and key in ['x', 'y', 'street_count']:
                    try:
                        data[key] = float(value)
                        node_conversions += 1
                    except (ValueError, TypeError):
                        # Some attributes may not be convertible, skip them
                        pass
        
        print(f"Converted {edge_conversions} edge attributes to numeric")
        print(f"Converted {node_conversions} node attributes to numeric")
        
        # ============================================================================
        # Connect to DuckDB and enable spatial extensions
        # ============================================================================
        print("\nStep 3: Connecting to DuckDB database...")
        state['db'] = duckdb.connect(db_path, read_only=False)
        
        # Install and load spatial extension for geographic queries
        state['db'].execute("INSTALL spatial; LOAD spatial;")
        print("DuckDB connected with spatial extensions enabled")
        
        # ============================================================================
        # Build schema information for AI query generation
        # ============================================================================
        print("\nStep 4: Analyzing database schema...")
        
        # Get graph statistics
        state['schema'] = {
            'nodes': state['graph'].number_of_nodes(),
            'edges': state['graph'].number_of_edges(),
            'tables': {}
        }
        
        # Get table schemas from database
        available_tables = []
        for table in ['nodes', 'edges', 'pois']:
            try:
                cols = state['db'].execute(f"PRAGMA table_info({table})").fetchdf()
                state['schema']['tables'][table] = cols['name'].tolist()
                available_tables.append(table)
                print(f"  Found table '{table}' with {len(cols)} columns")
            except:
                # Table might not exist in this database
                pass
        
        # ============================================================================
        # Print summary statistics
        # ============================================================================
        print("\n" + "="*80)
        print("ENVIRONMENT INITIALIZATION COMPLETE")
        print("="*80)
        console.print(f"[green]✓ Network loaded:[/green] {state['schema']['nodes']} nodes, {state['schema']['edges']} edges")
        console.print(f"[green]✓ Database ready:[/green] Tables: {list(state['schema']['tables'].keys())}")
        console.print(f"[green]✓ All attributes converted[/green] for NetworkX compatibility")
        
        # Sample some data for verification
        sample_node = list(state['graph'].nodes(data=True))[0]
        print(f"\nSample node data: {sample_node[0][:10]}... with {len(sample_node[1])} attributes")
        
        sample_edge = list(state['graph'].edges(data=True))[0]
        print(f"Sample edge data: ({sample_edge[0][:10]}..., {sample_edge[1][:10]}...) with {len(sample_edge[2])} attributes")
        
    except Exception as e:
        console.print(f"[red]✗ Initialization error: {e}[/red]")
        print(f"\nERROR: Failed to initialize environment")
        print(f"Details: {str(e)}")
        raise
    
    return True

# Initialize with default Chinatown NYC files
print("\nInitializing Chinatown, NYC street network for urban analysis...")
initialize_environment()

# ============================================================================
# III. Defining LLM-based Tools for the ReAct Agent
# ============================================================================

We define two specialized tools that leverage GPT-OSS-120b's capabilities:
1. **Network Analyst**: Generates and executes NetworkX code for graph algorithms
2. **Database Analyst**: Creates spatial SQL queries for POI analysis

Each tool uses GPT-OSS-120b's chain-of-thought reasoning to understand the query and generate appropriate code.

In [ ]:
@tool
def network_analyst(task: str) -> str:
    """
    Analyzes street network topology using NetworkX algorithms.
    Showcases GPT-OSS-120b's ability to generate complex graph analysis code.
    
    USE THIS TOOL WHEN:
    - Computing network metrics (centrality, connectivity, clustering)
    - Finding shortest paths between intersections
    - Analyzing network structure and topology
    - Calculating accessibility metrics
    - Identifying critical nodes or edges
    
    DO NOT USE WHEN:
    - Looking up specific places or POIs
    - Querying facility information
    - Needing exact addresses or names
    """
    global llm  # Use single global instance for performance
    
    print("\n" + "-"*60)
    print("⬢ NETWORK ANALYST TOOL ACTIVATED")
    print("-"*60)
    
    console.print(Panel(
        f"[bold]Task:[/bold] {task[:100]}...\n"
        f"[dim]Using GPT-OSS-120b to generate NetworkX analysis code[/dim]",
        title="⬢ Network Analyst",
        border_style="blue"
    ))
    
    # ============================================================================
    # Build comprehensive prompt for GPT-OSS-120b
    # We provide detailed context about the graph structure and requirements
    # ============================================================================
    
    prompt = f"""
You are an expert Python programmer specializing in NetworkX library for graph analysis.

CRITICAL GRAPH INFORMATION:
- Graph object 'G' is a MultiDiGraph with {state['schema']['nodes']} nodes and {state['schema']['edges']} edges
- G is already loaded - access it directly as 'G'
- ALL node IDs are STRINGS like '5340680144' - NEVER use integers
- Node attributes: 'y' (latitude), 'x' (longitude), 'street_count' (connections)
- Edge attributes: 'length' (meters as float), 'name' (street name), 'highway' (road type)

TASK: {task}

REQUIREMENTS:
1. Write Python code as ONE CONTINUOUS BLOCK (no blank lines)
2. Always use string node IDs: '5340680144' not 5340680144
3. Set FINAL_RESULT variable with your findings
4. Keep results concise (top 5-10 items, not all {state['schema']['nodes']} nodes)
5. Include lat/lon coordinates in results when relevant

EXAMPLE PATTERN:
# Calculate centrality
centrality = nx.degree_centrality(G)
top_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:5]
FINAL_RESULT = [{{
    "node_id": str(node_id),
    "centrality": round(value, 4),
    "lat": G.nodes[node_id].get('y', 0),
    "lon": G.nodes[node_id].get('x', 0)
}} for node_id, value in top_nodes]

Generate ONLY executable Python code. Add comments to explain your reasoning."""

    # ============================================================================
    # Attempt code generation with retry logic for robustness
    # ============================================================================
    
    for attempt in range(2):  # Allow one retry if needed
        if attempt > 0:
            console.print(f"[yellow]Retry {attempt}/1 with enhanced guidance...[/yellow]")
            prompt += "\n\nIMPORTANT: Node IDs must be strings! Use '5340680144' format."
        
        try:
            # Get GPT-OSS-120b to generate the analysis code
            print("\nInvoking GPT-OSS-120b for code generation...")
            console.print("\n[bold cyan]∵ GPT-OSS-120b Chain-of-Thought:[/bold cyan]")
            
            response = llm.invoke(prompt)
            
            # Extract and display the generated code
            code = response.content.strip().replace('```python', '').replace('```', '')
            
            # Display reasoning process (showcases model capability)
            console.print("[dim]" + "─" * 60 + "[/dim]")
            print("Generated NetworkX analysis code:")
            for line_num, line in enumerate(code.split('\n')[:20], 1):
                if line.strip().startswith('#'):
                    # Comments show the model's reasoning
                    console.print(f"[green]{line}[/green]")
                elif 'FINAL_RESULT' in line:
                    # Highlight the key output
                    console.print(f"[bold yellow]{line}[/bold yellow]")
                else:
                    console.print(f"[dim]{line}[/dim]")
            
            if len(code.split('\n')) > 20:
                console.print(f"[dim]... ({len(code.split('\n')) - 20} more lines of code)[/dim]")
            console.print("[dim]" + "─" * 60 + "[/dim]")
            
            # ============================================================================
            # Prepare code for execution
            # ============================================================================
            
            # Clean code - remove empty lines and imports
            lines = [line for line in code.split('\n') if line.strip()]
            code = '\n'.join(lines)
            
            # Remove import statements (already available in namespace)
            code = '\n'.join([line for line in code.split('\n') 
                            if not line.strip().startswith('import') and not line.strip().startswith('from')])
            
            # ============================================================================
            # Set up execution namespace with all required objects
            # CRITICAL: Graph must be in namespace for algorithms to work
            # ============================================================================
            
            exec_namespace = {
                'G': state['graph'],  # The graph object
                'nx': nx,            # NetworkX library
                'json': json,
                'math': math,
                'np': np,
                'pd': pd,
                # Built-in functions needed for analysis
                'str': str, 'float': float, 'int': int, 'round': round,
                'sorted': sorted, 'len': len, 'min': min, 'max': max,
                'list': list, 'dict': dict, 'set': set,
                'enumerate': enumerate, 'sum': sum,
                'FINAL_RESULT': None  # Will be set by the generated code
            }
            
            print("\nExecuting generated NetworkX code...")
            console.print("[bold green]→ Executing analysis...[/bold green]")
            
            # Execute with single namespace for variable persistence
            exec(code, exec_namespace, exec_namespace)
            
            # Extract the result
            result = exec_namespace.get('FINAL_RESULT')
            
            if result is not None:
                console.print(f"[green]✓ Analysis complete[/green]")
                print(f"Result type: {type(result)}")
                print(f"Result preview: {str(result)[:200]}...")
                return f"Analysis complete: {json.dumps(result, default=str)}"
            else:
                raise ValueError("FINAL_RESULT was not set by the generated code")
                
        except Exception as e:
            error_msg = str(e)
            console.print(f"[red]✗ Error: {error_msg}[/red]")
            print(f"\nExecution error details: {error_msg}")
            
            # Provide specific guidance for common errors
            if "is not in the graph" in error_msg:
                console.print("[yellow]Hint: Node IDs must be strings. Use '5340680144' not 5340680144[/yellow]")
                print("The model may have used integer node IDs instead of strings")
            
            if attempt == 0:
                print("Retrying with additional guidance...")
                continue
            else:
                return f"Network analysis failed: {error_msg}. Try rephrasing the query."
    
    return "Network analysis could not be completed after retries"

In [ ]:
@tool
def database_analyst(task: str) -> str:
    """
    Queries POIs and performs spatial database operations.
    Demonstrates GPT-OSS-120b's SQL generation and spatial reasoning.
    
    USE THIS TOOL WHEN:
    - Finding specific places (shops, hospitals, restaurants, etc.)
    - Calculating distances to/from POIs
    - Counting facilities by type
    - Spatial queries (within distance, nearest neighbor)
    - Filtering POIs by attributes
    
    DO NOT USE WHEN:
    - Computing graph algorithms
    - Analyzing network topology
    - Working only with intersection data
    """
    global llm  # Use single global instance for performance
    
    print("\n" + "-"*60)
    print("◉ DATABASE ANALYST TOOL ACTIVATED")
    print("-"*60)
    
    console.print(Panel(
        f"[bold]Task:[/bold] {task[:100]}...\n"
        f"[dim]Using GPT-OSS-120b to generate spatial SQL queries[/dim]",
        title="◉ Database Analyst",
        border_style="green"
    ))
    
    # ============================================================================
    # Build comprehensive prompt for SQL generation
    # Provide schema details and spatial function examples
    # ============================================================================
    
    prompt = f"""
You are an expert in DuckDB SQL with spatial extensions.

DATABASE SCHEMA:

Table 'nodes' (street intersections):
- node_id: VARCHAR (e.g., '5340680144')
- lat: DOUBLE (latitude)
- lon: DOUBLE (longitude)  
- street_count: INTEGER (number of connecting streets)
- geom: GEOMETRY (spatial point geometry)

Table 'pois' (points of interest / facilities):
- lat: DOUBLE (latitude)
- lon: DOUBLE (longitude)
- geom: GEOMETRY (spatial point geometry)
- amenity: VARCHAR (e.g., 'hospital', 'clinic', 'restaurant', 'school')
- building: VARCHAR (building type)
- name: VARCHAR (facility name)

SPATIAL FUNCTIONS AVAILABLE:
- ST_Distance(geom1, geom2): Calculate distance between geometries
- ST_Point(lon, lat): Create point geometry
- ST_Within(geom, distance): Check if within distance
- ST_Buffer(geom, distance): Create buffer around geometry

TASK: {task}

REQUIREMENTS:
1. Write a single, efficient SQL query
2. For medical facilities use: WHERE amenity IN ('hospital', 'clinic', 'health_center')
3. Order results appropriately (usually by distance or importance)
4. Limit results to reasonable counts (5-20 rows)
5. Include comments explaining your approach

Provide ONLY the SQL query with comments."""

    try:
        # ============================================================================
        # Get GPT-OSS-120b to generate the SQL query
        # ============================================================================
        
        print("\nInvoking GPT-OSS-120b for SQL generation...")
        console.print("\n[bold cyan]∵ GPT-OSS-120b SQL Generation:[/bold cyan]")
        
        response = llm.invoke(prompt)
        
        # Extract SQL from response
        sql = response.content.strip().replace('```sql', '').replace('```', '')
        
        # Display SQL with syntax highlighting for readability
        console.print("[dim]" + "─" * 60 + "[/dim]")
        print("Generated SQL query:")
        for line in sql.split('\n'):
            if line.strip().startswith('--'):
                # SQL comments show reasoning
                console.print(f"[green]{line}[/green]")
            elif any(keyword in line.upper() for keyword in ['SELECT', 'FROM', 'WHERE', 'ORDER', 'LIMIT', 'JOIN', 'GROUP']):
                # Highlight SQL keywords
                console.print(f"[bold blue]{line}[/bold blue]")
            else:
                console.print(f"[dim]{line}[/dim]")
        console.print("[dim]" + "─" * 60 + "[/dim]")
        
        # ============================================================================
        # Execute the generated SQL query
        # ============================================================================
        
        print("\nExecuting SQL query against DuckDB...")
        console.print("[bold green]→ Executing query...[/bold green]")
        
        result_df = state['db'].execute(sql).fetchdf()
        
        # ============================================================================
        # Process and format results
        # ============================================================================
        
        console.print(f"[green]✓ Query returned {len(result_df)} rows[/green]")
        print(f"\nQuery execution successful")
        print(f"Columns returned: {list(result_df.columns)}")
        
        if len(result_df) == 0:
            print("No results found - the requested amenity type may not exist in this area")
            return "No results found. The requested amenity type may not exist in this area."
        elif len(result_df) > 20:
            print(f"Large result set - showing top 10 of {len(result_df)} rows")
            return f"Found {len(result_df)} results. Top 10:\n{result_df.head(10).to_string()}"
        else:
            print(f"Returning all {len(result_df)} results")
            return f"Results ({len(result_df)} rows):\n{result_df.to_string()}"
            
    except Exception as e:
        error_msg = str(e)
        console.print(f"[red]✗ Query error: {error_msg}[/red]")
        print(f"\nSQL execution error: {error_msg}")
        
        # Provide helpful error context
        if "no such table" in error_msg.lower():
            print("The requested table does not exist in the database")
        elif "no such column" in error_msg.lower():
            print("The query references a column that doesn't exist")
        
        return f"Database query failed: {error_msg}. Try simplifying the query."

In [ ]:
def analyze(query: str):
    """
    Process urban analysis queries using GPT-OSS-120b's advanced reasoning.
    Orchestrates multiple tools to provide comprehensive insights.
    
    This is the main entry point for all analysis queries. It uses a ReAct agent
    to break down complex questions and select appropriate tools.
    
    Args:
        query: Natural language question about the street network
    
    Returns:
        str: Analysis results with insights and recommendations
    """
    global llm  # Use single global instance for performance
    
    # ============================================================================
    # Display the query with formatting
    # ============================================================================
    
    console.print("\n" + "="*70)
    console.print(Panel.fit(
        f"[bold cyan]{query}[/bold cyan]",
        title="◐ DreamStreets Analysis Query",
        border_style="cyan",
        box=box.DOUBLE
    ))
    console.print("="*70)
    
    print(f"\nProcessing query: {query[:100]}...")
    print(f"Graph info: {state['schema']['nodes']} nodes, {state['schema']['edges']} edges")
    print(f"Database tables: {list(state['schema']['tables'].keys())}")
    
    # ============================================================================
    # Set up the tools for the ReAct agent
    # ============================================================================
    
    tools = [network_analyst, database_analyst]
    
    # ============================================================================
    # Build enhanced context for the agent
    # This helps GPT-OSS-120b understand the available data and tools
    # ============================================================================
    
    enhanced_query = f"""
SYSTEM CONTEXT:
- Street network graph 'G' loaded with {state['schema']['nodes']} nodes and {state['schema']['edges']} edges
- Database contains: {list(state['schema']['tables'].keys())} tables
- All node IDs are STRINGS (e.g., '5340680144')
- Numeric attributes (length, x, y) are floats

AVAILABLE TOOLS:
1. network_analyst: Graph algorithms, centrality, paths, network metrics
2. database_analyst: Find places, count facilities, spatial queries

USER QUERY: {query}

Provide a comprehensive analysis with specific numbers and actionable insights.
Use chain-of-thought reasoning to break down the problem and select appropriate tools.
"""
    
    console.print("[bold]∵ GPT-OSS-120b analyzing query and selecting tools...[/bold]\n")
    print("\nInitializing ReAct agent with GPT-OSS-120b...")
    
    # ============================================================================
    # Create the ReAct agent with our tools
    # ============================================================================
    
    agent = create_react_agent(llm, tools)
    
    start_time = time.time()
    
    try:
        print("Invoking agent to process query...")
        print("This may take a moment as GPT-OSS-120b reasons through the problem...")
        
        # Execute the agent with recursion limit for safety
        result = agent.invoke(
            {"messages": [HumanMessage(content=enhanced_query)]},
            config={"recursion_limit": 25}
        )
        
        # Extract the final answer
        final_answer = result["messages"][-1].content
        
    except Exception as e:
        if "recursion limit" in str(e).lower():
            print("\nQuery too complex - reached maximum reasoning depth")
            final_answer = "⚠ Analysis reached maximum complexity. Try breaking down your query into simpler parts."
        else:
            print(f"\nAgent execution error: {str(e)}")
            final_answer = f"✗ Analysis error: {str(e)}"
    
    elapsed = time.time() - start_time
    
    # ============================================================================
    # Display results with formatting
    # ============================================================================
    
    console.print("\n" + "="*70)
    console.print("[bold green]◎ ANALYSIS COMPLETE[/bold green]")
    console.print("="*70)
    
    # Use Markdown for rich formatting of results
    display(IPMarkdown(final_answer))
    
    # Display performance metrics
    print("\n" + "-"*70)
    console.print(f"[dim]◷ Analysis time: {elapsed:.1f} seconds using GPT-OSS-120b[/dim]")
    console.print(f"[dim]⚙ Graph: {state['schema']['nodes']} nodes | Database: {len(state['schema']['tables'])} tables[/dim]")
    print("-"*70)
    
    return final_answer

# ============================================================================
# IV. Urban Planning Analysis - Chinatown, NYC
# ============================================================================

We begin with dense urban network analysis for business optimization and public safety in NYC's Chinatown. These queries demonstrate how GPT-OSS-120b can help with real-world urban planning decisions.

### Query 1: Optimal Coffee Shop Location

Finding the best intersection for a new business based on network centrality metrics.

In [ ]:
# ============================================================================
# Query 1: Business location optimization using betweenness centrality
# This metric identifies intersections that appear on the most shortest paths
# ============================================================================

result = analyze(
    "I want to open a coffee shop. Which intersection has the highest foot traffic "
    "based on betweenness centrality? Show me the top 5 locations with coordinates."
)

### Query 2: Critical Infrastructure Analysis

Identifying network bottlenecks that would cause maximum disruption if blocked.

In [ ]:
# ============================================================================
# Query 2: Infrastructure vulnerability assessment
# Finding articulation points - nodes whose removal disconnects the network
# ============================================================================

result = analyze(
    "Which intersection is the most critical bottleneck in the network? "
    "Find nodes whose removal would most increase distances between other nodes."
)

### Query 3: Healthcare Accessibility

Identifying areas with poor access to medical facilities - critical for public health planning.

In [ ]:
# ============================================================================
# Query 3: Healthcare desert identification
# Finding areas furthest from medical facilities for mobile clinic placement
# ============================================================================

result = analyze(
    "Find the 3 intersections that are furthest from any medical facility. "
    "These represent healthcare deserts that need mobile clinics."
)

### Query 4: Food Truck Optimization

Strategic placement of mobile vendors to maximize coverage while avoiding competition.

In [ ]:
# ============================================================================
# Query 4: Mobile vendor placement optimization
# Using dominating set algorithms to maximize coverage
# ============================================================================

result = analyze(
    "Where should 3 food trucks position themselves to maximize coverage? "
    "Find intersections that together reach the most nodes within 400m walking distance "
    "while avoiding existing restaurants."
)

# ============================================================================
# V. Humanitarian Response Analysis - Cox's Bazar Refugee Camp
# ============================================================================

**Context:** The Kutupalong-Balukhali expansion site in Cox's Bazar, Bangladesh is the world's largest refugee settlement with approximately 1 million Rohingya refugees. During monsoon season, flooding isolates communities and disrupts supply chains.

**Mission:** Use GPT-OSS-120b's reasoning to optimize emergency resource placement and identify vulnerable areas.

This section demonstrates the humanitarian impact potential required for the "For Humanity" prize category.

In [ ]:
# ============================================================================
# Switch to Cox's Bazar refugee camp street network
# This network represents the pathways through the camp
# ============================================================================

print("\n" + "="*80)
print("SWITCHING TO HUMANITARIAN CONTEXT")
print("Loading Cox's Bazar refugee camp network...")
print("="*80)

initialize_environment('coxs_bazar.graphml', 'coxs_bazar.duckdb')

print("\nContext: World's largest refugee camp with ~1 million residents")
print("Challenge: Monsoon flooding isolates communities")
print("Mission: Strategic emergency resource placement")

### Query 5: Emergency Evacuation Centers

Optimal placement of evacuation centers for rapid access during emergencies.

In [ ]:
# ============================================================================
# Query 5: Evacuation center placement
# Using closeness centrality to find locations with shortest average distances
# ============================================================================

result = analyze(
    "Where should we build emergency evacuation centers? Find the 3 intersections "
    "with highest closeness centrality that can quickly reach all areas of the camp."
)

### Query 6: Flood Response Priority

Identifying critical points that would isolate communities if flooded.

In [ ]:
# ============================================================================
# Query 6: Flood vulnerability assessment
# Finding articulation points that would split the network if removed
# ============================================================================

result = analyze(
    "Which intersections are articulation points that would isolate communities if flooded? "
    "These need elevated platforms for supply distribution during monsoons."
)

### Query 7: Water Access Vulnerability

Critical for cholera prevention - identifying areas at risk of losing water access.

In [ ]:
# ============================================================================
# Query 7: Water access vulnerability mapping
# Critical for disease prevention in refugee camps
# ============================================================================

result = analyze(
    "Identify areas that would lose water access if main roads flood. "
    "Find intersections where closure cuts off the most people from water points. "
    "Critical for cholera prevention."
)

### Query 8: Medical Emergency Routes

Ensuring safe passage for medical emergencies, especially at night.

In [ ]:
# ============================================================================
# Query 8: Emergency medical route optimization
# Pregnant women and sick children need safe routes at all hours
# ============================================================================

result = analyze(
    "Which paths need lighting for night medical emergencies? "
    "Find shortest routes from the 5 most populated areas to the nearest hospital. "
    "Pregnant women and sick children can't wait until morning."
)

# ============================================================================
# VI. Summary and Impact
# ============================================================================

## DreamStreets Achievements

**DreamStreets** successfully demonstrates GPT-OSS-120b's advanced capabilities for real-world impact:

### Technical Innovation
- ✓ **Chain-of-thought reasoning** for complex spatial problems
- ✓ **Code generation** for NetworkX graph algorithms  
- ✓ **SQL synthesis** for spatial database queries
- ✓ **Multi-tool orchestration** via ReACT agents
- ✓ **Complete offline operation** - no internet required after setup

### Real-World Applications
- **Urban Planning**: Business location optimization, infrastructure vulnerability
- **Public Health**: Healthcare desert identification, emergency access routes
- **Humanitarian Response**: Refugee camp management, flood preparedness
- **Emergency Management**: Evacuation planning, resource distribution

### Hackathon Category Alignment
- **Best Local Agent**: Runs completely offline with GPT-OSS-120b via Ollama
- **For Humanity**: Direct application to refugee camp management and disaster response
- **Best Overall**: Showcases model's reasoning capabilities on critical real-world problems

### Impact Metrics
- Analyzes networks with thousands of nodes in seconds
- Provides actionable insights for decision makers
- Scalable to any geographic region with OSM data
- No ongoing costs after initial setup (fully local)

---

_Powered by GPT-OSS-120b - OpenAI's most advanced open-source model_

_Building on AskStreets foundation - Winner of ArangoDB GraphRAG Hackathon_